In [2]:
import numpy as np
import plotly.graph_objects as go

# Constants
c = 3e8  # Speed of light (m/s)
fc = 28e9  # Carrier frequency (Hz)
wavelength = c / fc  # Wavelength (meters)
d = wavelength / 2  # Distance between antenna elements (half-wavelength spacing)
Nx, Ny = 8, 8  # Number of antenna elements in the x and y dimensions (URA)
codebook_size_azimuth = 32  # Number of beams in the azimuth direction
codebook_size_elevation = 16  # Number of beams in the elevation direction
z_max = 0  # Height of the base station on the z-axis

# Generate 3D steering vectors for a URA (Uniform Rectangular Array)
def steering_vector_3d(azimuth_rad, elevation_rad, Nx, Ny, d, wavelength):
    nx = np.arange(Nx)  # Antenna elements in the x-axis
    ny = np.arange(Ny)  # Antenna elements in the y-axis
    steering_x = np.exp(-1j * 2 * np.pi * d * nx * np.sin(elevation_rad) * np.cos(azimuth_rad) / wavelength)
    steering_y = np.exp(-1j * 2 * np.pi * d * ny * np.sin(azimuth_rad) / wavelength)
    return np.kron(steering_x, steering_y)

# Generate the codebook of 3D beamforming vectors
def generate_3d_codebook(Nx, Ny, d, wavelength, codebook_size_azimuth, codebook_size_elevation):
    codebook = []
    azimuth_angles = np.linspace(-np.pi, np.pi, codebook_size_azimuth)  # Full azimuth angles
    elevation_angles = np.linspace(0, -np.pi/2, codebook_size_elevation)  # Lower hemisphere
    for elevation in elevation_angles:
        for azimuth in azimuth_angles:
            codebook.append(steering_vector_3d(azimuth, elevation, Nx, Ny, d, wavelength))
    return np.array(codebook), azimuth_angles, elevation_angles

# User location in the lower hemisphere
user_azimuth_deg = 30  # User's azimuth angle
user_elevation_deg = -26  # User's elevation angle
user_azimuth_rad = np.deg2rad(user_azimuth_deg)
user_elevation_rad = np.deg2rad(user_elevation_deg)

# Create the 3D codebook
codebook, azimuth_angles, elevation_angles = generate_3d_codebook(Nx, Ny, d, wavelength, codebook_size_azimuth, codebook_size_elevation)

# Channel between the base station and the user
h = steering_vector_3d(user_azimuth_rad, user_elevation_rad, Nx, Ny, d, wavelength)  # User's channel vector

# Calculate received signal power for each beam in the 3D codebook
received_power = []
for beam in codebook:
    received_signal = np.dot(beam.conj().T, h) #-----> for simulating receiving signal dot product is done b/w weight vectors from code book and actual needed weight vector
    power = np.abs(received_signal) ** 2
    received_power.append(power)

# Reshape received power to 2D grid
received_power = np.array(received_power).reshape((codebook_size_elevation, codebook_size_azimuth))

best_beam_idx = np.unravel_index(np.argmax(received_power), received_power.shape)
best_azimuth_angle = np.rad2deg(azimuth_angles[best_beam_idx[1]])
best_elevation_angle = np.rad2deg(elevation_angles[best_beam_idx[0]])

# Prepare data for Plotly
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=[0], y=[0], z=[z_max],
    mode='markers',
    marker=dict(size=10, color='red'),
    name='Base Station'
))


beam_length = 30
for i, elevation in enumerate(elevation_angles):
    for j, azimuth in enumerate(azimuth_angles):
        x = beam_length * np.cos(elevation) * np.sin(azimuth)
        y = beam_length * np.cos(elevation) * np.cos(azimuth)
        z = beam_length * np.sin(elevation)

        fig.add_trace(go.Scatter3d(
            x=[0, x], y=[0, y], z=[z_max, z],
            mode='lines',
            line=dict(color='blue', width=2),
            name='Beam',
            opacity=0.5
        ))

        # Bullet markers at the end of each beam
        fig.add_trace(go.Scatter3d(
            x=[x], y=[y], z=[z],
            mode='markers',
            marker=dict(size=5, color='blue'),
            name='End of Beam'
        ))

# User position
user_x = beam_length * np.cos(user_elevation_rad) * np.sin(user_azimuth_rad)
user_y = beam_length * np.cos(user_elevation_rad) * np.cos(user_azimuth_rad)
user_z = beam_length * np.sin(user_elevation_rad)  # User is below the base station
fig.add_trace(go.Scatter3d(
    x=[user_x], y=[user_y], z=[user_z],
    mode='markers',
    marker=dict(size=10, color='green'),
    name='User'
))

# Highlight the best beam
best_x = beam_length * np.cos(np.deg2rad(best_elevation_angle)) * np.sin(np.deg2rad(best_azimuth_angle))
best_y = beam_length * np.cos(np.deg2rad(best_elevation_angle)) * np.cos(np.deg2rad(best_azimuth_angle))
best_z = beam_length * np.sin(np.deg2rad(best_elevation_angle))
fig.add_trace(go.Scatter3d(
    x=[0, best_x], y=[0, best_y], z=[z_max, best_z],
    mode='lines',
    line=dict(color='red', width=4),
    name='Best Beam',
    opacity=1 
))

# Layout settings
fig.update_layout(
    scene=dict(
        xaxis_title='X (Azimuth)',
        yaxis_title='Y (Azimuth)',
        zaxis_title='Z (Elevation)',
    ),
    title=f'3D Beamforming: User at {user_azimuth_deg}°, {user_elevation_deg}°',
    showlegend=True
)

fig.show()

print(f'Best beam azimuth angle: {best_azimuth_angle:.2f} degrees')
print(f'Best beam elevation angle: {best_elevation_angle:.2f} degrees')


Best beam azimuth angle: 29.03 degrees
Best beam elevation angle: -24.00 degrees
